# 3장: 엑셀파일을 만들게 시키다. 여기서 시간을 쓰다.

# 3.1 엑셀 통합 문서 내부 살펴보기

In [1]:
#!/usr/bin/env python3
from xlrd import open_workbook

input_file = '/Users/henry/Desktop/sales_2013.xlsx'

workbook = open_workbook(input_file)
print('Number of worksheets:', workbook.nsheets)
for worksheet in workbook.sheets():
    print("Worksheet name:", worksheet.name, "\tRows:", worksheet.nrows, "\tColumns:", worksheet.ncols)

Number of worksheets: 3
Worksheet name: january_2013 	Rows: 7 	Columns: 5
Worksheet name: february_2013 	Rows: 7 	Columns: 5
Worksheet name: march_2013 	Rows: 7 	Columns: 5


1. nrows: 로우의 갯수 확인
2. ncols: 컬럼의 갯수 확인

# 3.2 단일 워크시트 처리

## 3.2.1 엑셀파일 읽기 및 쓰기

In [5]:
#xlrd 및 xlwt 모듈을 사용한 기본 파이썬
#!/usr/bin/env python3
import sys
from xlrd import open_workbook
from xlwt import Workbook

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_good.xls'

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('jan_2013_output')

with open_workbook(input_file) as workbook:
    worksheet = workbook.sheet_by_name('january_2013')
    for row_index in range(worksheet.nrows):
        for column_index in range(worksheet.ncols):
            output_worksheet.write(row_index, column_index, worksheet.cell_value(row_index, column_index))
            print(worksheet.cell_value(row_index, column_index))

Customer ID
Customer Name
Invoice Number
Sale Amount
Purchase Date
1234.0
John Smith
100-0002
1200.0
41275.0
2345.0
Mary Harrison
100-0003
1425.0
41280.0
3456.0
Lucy Gomez
100-0004
1390.0
41285.0
4567.0
Rupert Jones
100-0005
1257.0
41282.0
5678.0
Jenny Walters
100-0006
1725.0
41298.0
6789.0
Samantha Donaldson
100-0007
1995.0
41305.0


1. Workbook: 엑셀 빈 파일을 지정한다.
2. 얘는 시트를 생성한거다.
3. 0,1,2,3,4,5,6하고 칼럼을 돈다.
4. sheet_by_name : 시트 목록을 출력 / sheet_by_index(): 씨트 인덱스를 출력
5. 로우와 칼럼을 돌아서 다 채워넣어라아

In [6]:
#날짜 형식 할당
#!/usr/bin/env python3
import sys
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_hi.xls'

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('jan_2013_output')

with open_workbook(input_file) as workbook:
    worksheet = workbook.sheet_by_name('january_2013')
    for row_index in range(worksheet.nrows):
        row_list_output = []
        for col_index in range(worksheet.ncols):
            if worksheet.cell_type(row_index, col_index) == 3:
                date_cell = xldate_as_tuple(worksheet.cell_value(row_index, col_index), workbook.datemode)
                date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                row_list_output.append(date_cell)
                output_worksheet.write(row_index, col_index, date_cell)
            else:
                non_date_cell = worksheet.cell_value(row_index, col_index)
                row_list_output.append(non_date_cell)
                output_worksheet.write(row_index, col_index, non_date_cell)
print(output_workbook)

In [5]:
#판다스 그냥 엑셀 읽어
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_shit.xls'

data_frame = pd.read_excel(input_file, sheetname='january_2013')

writer = pd.ExcelWriter(output_file)
data_frame.to_excel(writer, sheet_name='jan_13_output', index=False)
writer.save()

### 위에꺼 그냥 출력해봄

In [1]:
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_shit.xls'

data_frame = pd.read_excel(input_file, sheetname='january_2013')
print(data_frame)

   Customer ID       Customer Name Invoice Number  Sale Amount Purchase Date
0         1234          John Smith       100-0002         1200    2013-01-01
1         2345       Mary Harrison       100-0003         1425    2013-01-06
2         3456          Lucy Gomez       100-0004         1390    2013-01-11
3         4567        Rupert Jones       100-0005         1257    2013-01-08
4         5678       Jenny Walters       100-0006         1725    2013-01-24
5         6789  Samantha Donaldson       100-0007         1995    2013-01-31


1. 그냥 인풋파일에 시트네임 저거를 가져와서 뽑았다. 이것도 만족스럽다.

# 3.2.2 특정 행 필터링 하기 - 특정 조건을 충족하는 행의 필터링

In [13]:
# 기본 파이썬 : sale_amount가 $1,400.00보다 큰 하위 데이터셋으로 선택
#!/usr/bin/env python3
import sys
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_hell.xls'

output_wirkbook = Workbook()
output_worksheet = output_workbook.add_sheet('jan_2013_output1')

sale_amount_column_index = 3
with open_workbook(input_file) as workbook:
    worksheet = workbook.sheet_by_name('january_2013')
    data = []
    header = worksheet.row_values(0)
    data.append(header)
    for row_index in range(1,worksheet.nrows):
        row_list = []
        sale_amount = worksheet.cell_value(row_index, sale_amount_column_index)
        if sale_amount > 1400.0:
            for column_index in range(worksheet.ncols):
                cell_value = worksheet.cell_value(row_index, column_index)
                cell_type = worksheet.cell_type(row_index, column_index)
                if cell_type == 3:
                    date_cell = xldate_as_tuple(cell_value,workbook.datemode)
                    date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                    row_list.append(date_cell)
                else:
                    row_list.append(cell_value)
        if row_list:
            data.append(row_list)
            
    for list_index, output_list in enumerate(data):
        for element_index, element in enumerate(output_list):
            output_worksheet.write(list_index, element_index, element)
            
print(row_index)

6


In [7]:
#팬더스
#!/usr/bin/env python3
import pandas as pd
import sys

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_hing.xls'

data_frame = pd.read_excel(input_file, 'january_2013', index_col=None) 
#파일 불러올 때 index 지정해주기 : index_col
data_frame_value_meets_condition = data_frame[data_frame['Sale Amount'].astype(float) > 1400.0] #astype이 뭐지?

writer = pd.ExcelWriter(output_file)
data_frame_value_meets_condition.to_excel(writer, sheet_name='jan_13_output', index=False)
print(data_frame_value_meets_condition)

   Customer ID       Customer Name Invoice Number  Sale Amount Purchase Date
1         2345       Mary Harrison       100-0003         1425    2013-01-06
4         5678       Jenny Walters       100-0006         1725    2013-01-24
5         6789  Samantha Donaldson       100-0007         1995    2013-01-31


1. january_2013 출력을 data_frame 으로 지정
2. Sale Amount가 1400.0 초과인 것을 data_frame_value_meets_condition 으로 지정
3. 뽑아라. jan_13_output 시트로.(일단 프린트도 해봄). writer 부터는 무시해도 된다. 안 뽑을 꺼니까.

# 특정 집합의 값을 포함하는 행의 필터링

In [7]:
#!/usr/bin/env python3
import sys
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_gg.xls'

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('jan_2013_output')

important_dates = ['01/24/2013', '01/31/2013']

purchase_date_column_index = 4
with open_workbook(input_file) as workbook:
    worksheet = workbook.sheet_by_name('january_2013')
    data = []
    header = worksheet.row_values(0)
    data.append(header)
    for row_index in range(1, worksheet.nrows):
        purchase_datetime = xldate_as_tuple(worksheet.cell_value(row_index, purchase_date_column_index),workbook.datemode)
        purchase_date = date(*purchase_datetime[0:3]).strftime('%m/%d/%Y')
        row_list = []
        if purchase_date in important_dates:
            for column_index in range(worksheet.ncols):
                cell_value = worksheet.cell_value(row_index,column_index)
                cell_type = worksheet.cell_type(row_index,column_index)
                if cell_type == 3:
                    date_cell = xldate_as_tuple(cell_value,workbook.datemode)
                    date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                    row_list.append(date_cell)
                else:
                    row_list.append(cell_value)
        if row_list:
            data.append(row_list)
    
    for list_index, output_list in enumerate(data):
        for element_index, element in enumerate(output_list):
            output_worksheet.write(list_index, element_index, element)
            
output_workbook.save(output_file)

#enumerate = 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력받아 인덱스값에 포함시키는 역할. 대부분 순서를 내야하기 때문에 for문과 같이 쓰임.

In [3]:
# 01/24/2014 혹은 01/31/2013인 행을 필터링
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_haddd.xls'

data_frame = pd.read_excel(input_file, 'january_2013', index_col=None)

important_dates = ['01/24/2013', '01/31/2013']
data_frame_value_in_set = data_frame[data_frame['Purchase Date'].isin(important_dates)]
print(data_frame_value_in_set)
writer = pd.ExcelWriter(output_file)
data_frame_value_in_set.to_excel(writer, sheet_name='jan_13_output1', index=True)

writer.save()

   Customer ID       Customer Name Invoice Number  Sale Amount Purchase Date
4         5678       Jenny Walters       100-0006         1725    2013-01-24
5         6789  Samantha Donaldson       100-0007         1995    2013-01-31


1. isin : 포함되어 있는가? 라는 뜻이랍니다.
2. january_2013을 읽었다.
3. 변수로 저 두 날짜를 지정했다.
4. 두 날짜가 포함된 Purchase Date를 data_frame_value_in_set으로 지정
5. dat_frame_value_in_set을 프린트하거라.

## 패턴을 활용한 필터링

In [6]:
# Costomer Name 열이 J로 시작하는 것 필터링
#!/usr/bin/env python3
import re
import sys
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_pigon.xls'

output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('jan_2013_output')

pattern = re.compile(r'(?P<my_pattern>^J.*)')

customer_name_column_index = 1
with open_workbook(input_file) as workbook:
    worksheet = workbook.sheet_by_name('january_2013')
    data = []
    header = worksheet.row_values(0)
    data.append(header)
    for row_index in range(1, worksheet.nrows):
        row_list = []
        if pattern.search(worksheet.cell_value(row_index, customer_name_column_index)):
            for column_index in range(worksheet.ncols):
                cell_value = worksheet.cell_value(row_index,column_index)
                cell_type = worksheet.cell_type(row_index, column_index)
                if cell_type == 3:
                    date_cell = xldate_as_tuple(cell_value,workbook.datemode)
                    date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                    row_list.append(date_cell)
                else:
                    row_list.append(cell_value)
            if row_list:
                data.append(row_list)
                
        for list_index, output_list in enumerate(data):
            for element_index, element in enumerate(output_list):
                output_worksheet.write(list_index, element_index, element)
                
print(output_worksheet)

Exception: Attempt to overwrite cell: sheetname='jan2013_output' rowx=0 colx=0

In [7]:
# Costomer Name 열이 J로 시작하는 것 필터링
#!/usr/bin/env python3
import pandas as pd
import sys

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_kk.xls'

data_frame = pd.read_excel(input_file, 'january_2013', index_col=None)

data_frame_value_matches_pattern = data_frame[data_frame['Customer Name'].str.startswith("J")]

writer = pd.ExcelWriter(output_file)
data_frame_value_matches_pattern.to_excel(writer, sheet_name='jan_13_output', index=False)
print(data_frame_value_matches_pattern)

   Customer ID  Customer Name Invoice Number  Sale Amount Purchase Date
0         1234     John Smith       100-0002         1200    2013-01-01
4         5678  Jenny Walters       100-0006         1725    2013-01-24


1. startswith("J") : J로 시작하는 문자열을 찾는다.

## 심심해서 해본 startswith, endswith

In [13]:
'adgje'.startswith("adg")

True

In [11]:
'adgje'.endswith('e')

True

# 3.2.3 특정 열 선택하기

In [8]:
#특정 열을 선택하는 방법
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_qq.xls'

data_frame = pd.read_excel(input_file, 'january_2013', index_col=None)

data_frame_column_by_index = data_frame.iloc[:,[1,4]]

writer = pd.ExcelWriter(output_file)
data_frame_column_by_index.to_excel(writer, sheet_name='jan_13_output', index=False)  
print(data_frame_column_by_index)

        Customer Name Purchase Date
0          John Smith    2013-01-01
1       Mary Harrison    2013-01-06
2          Lucy Gomez    2013-01-11
3        Rupert Jones    2013-01-08
4       Jenny Walters    2013-01-24
5  Samantha Donaldson    2013-01-31


1. XlsxWriter를 엔진으로 사용하여 팬더 엑셀 작성자를 만듭니다.
2. to_excel: 데이터 프레임을 XlsxWriter Excel 개체로 변환합니다.
3. iloc를 통해 1번, 4번에 해당되는 데이터만을 가져왔다. 1번은 네임, 4번은 데이트.

## 열 헤더를 사용하여 특정 열 선택하기

In [9]:
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_ww.xls'

data_frame = pd.read_excel(input_file, 'january_2013', index_col=None)

data_frame_column_by_name = data_frame.loc[:, ['Customer ID', 'Purchase Date']]

writer = pd.ExcelWriter(output_file)
data_frame_column_by_name.to_excel(writer, sheet_name='jan_13_output', index=False)
print(data_frame_column_by_name)

   Customer ID Purchase Date
0         1234    2013-01-01
1         2345    2013-01-06
2         3456    2013-01-11
3         4567    2013-01-08
4         5678    2013-01-24
5         6789    2013-01-31


1. 위와 같은 결과, 위는 iloc를 활용했고, 아래는 loc를 활용. iloc와 loc는 안영님이 엄청 잘 정리해주심.

# 3.3 통합 문서의 모든 워크시트 읽기

## 모든 워크시트에서 특정 행 필터링하기

In [24]:
#모든 워크시트에서 Sale Amount 열의 데이터 값이 $2,000.00 이상인 모든 행을 필터링
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_buza.xls'

data_frame = pd.read_excel(input_file, sheetname=None, index_col=None) 

row_output = []
for worksheet_name, data in data_frame.items():
    row_output.append(data[data['Sale Amount'].replace('$','').replace(',','').astype(float) > 2000.0])
    filtered_row = pd.concat(row_output, axis=0, ignore_index=True)
    
writer = pd.ExcelWriter(output_file)
filtered_row.to_excel(writer, sheet_name='sale_amount_gt2000', index=False)
print(row_output)

[Empty DataFrame
Columns: [Customer ID, Customer Name, Invoice Number, Sale Amount, Purchase Date]
Index: [],    Customer ID Customer Name Invoice Number  Sale Amount Purchase Date
2         7654  Roger Lipney       100-0010         2135    2013-02-15,    Customer ID  Customer Name Invoice Number  Sale Amount Purchase Date
3         6543     Rachel Paz       100-0017         2042    2013-03-22
5         4321  Susan Wallace       100-0019         2280    2013-03-30]


1. sheetname을 None으로 하면 한 번에 통합 문서의 모든 워크시트를 읽는다.
2. append: 리스트의 맨 마지막에 새로운 요소를 추가하는 것
3. replace("찾을값", "바꿀값", [바꿀횟수])
4. concat : concatenation. 연결을 뜻함.
5. axis가 뭔데?

## 모든 워크시트에서 특정 열 선택하기

In [25]:
#!/usr/bin/env python3
import pandas as pd

input_file = '/Users/henry/Desktop/sales_2013.xlsx'
output_file = '/Users/henry/Desktop/sales_2013_wa.xls'

data_frame = pd.read_excel(input_file, sheetname =None, index_col=None)

column_output = []
for worksheet_name, data in data_frame.items(): 
    column_output.append(data.loc[:, ['Customer Name', 'Sale Amount']])
selected_columns = pd.concat(column_output, axis=0, ignore_index=False)

writer = pd.ExcelWriter(output_file)
selected_columns.to_excel(writer, sheet_name='selected_columns_all_worksheets', index=False)
print(selected_columns)

        Customer Name  Sale Amount
0          John Smith         1200
1       Mary Harrison         1425
2          Lucy Gomez         1390
3        Rupert Jones         1257
4       Jenny Walters         1725
5  Samantha Donaldson         1995
0       Daniel Farber         1115
1         Laney Stone         1367
2        Roger Lipney         2135
3       Thomas Haines         1346
4         Anushka Vaz         1560
5       Harret Cooper         1852
0          John Smith         1350
1           Tony Song         1167
2       Maty Harrison         1789
3          Rachel Paz         2042
4          Lucy Gomez         1511
5       Susan Wallace         2280


1. 딕셔너리의 Key와 Value의 쌍으로 구성된 아이템들의 리스트를 얻는 것
2. loc를 사용하여 각 워크시트의 특정 열을 선택하여 필터링된 데이터프레임의 리스트를 만들고, 데이터프레임들을 하나의 최종적인 데이터프레임으로 합친다.
3. ignore_index가 먹은 건가??

# 이 책은 정말 입문이라는 단어를 지워버려야 한다.
# 이걸 안 보고 내가 할 수 있을까? 라는 생각이 든다.
# 그래도 이제 쪼금~~ 읽을 수는 있을 것 같다.(판다스만)
# 반복숙달만이..